In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import re
import requests
from io import BytesIO
from lxml import etree
sys.path.insert(0,"..")
# from src.madamira.generate_config import generate_config, feats
from src.madamira.analyse import analyse_standalone, analyse_server, write_xml, load_analysis, parse_analyser_output, generate_analyser_input
from src.predict import translit_rules
from data.make_dataset import recompose, tokenize_skiphyph


In [2]:
home = Path(os.getcwd()).parent
home

PosixPath('/Users/fae211/ba3sasah/LOC_transcribe')

In [3]:
dev = pd.read_csv(f'{home}/data/processed/dev.tsv',delimiter='\t')
# devsents = list(dev['ar'].values)[35742]

In [11]:
dev[dev['sentID']==35742]['ar'].values

array(['علاقبند، فاطمه.'], dtype=object)

In [ ]:
['علاقبند، فاطمه.']

In [12]:
testanalyses

[#WORDS#: ['علاقبند', '،', 'فاطمه', '.']; #AN#: {'علاقبند': {'bw': 'ElAqbnd/NOUN_PROP', 'gloss': 'NO_ANALYSIS', 'diac': 'علاقبند', 'lemma': 'علاقبند_0', 'asp': 'na', 'cas': 'u', 'gen': 'm', 'mod': 'na', 'num': 's', 'per': 'na', 'stt': 'i', 'vox': 'na', 'pos': 'noun_prop', 'prc0': '0', 'prc1': '0', 'prc2': '0', 'prc3': '0', 'enc0': '0'}, '،': {'bw': '،/NOUN_PROP', 'gloss': 'NO_ANALYSIS', 'diac': '،', 'lemma': '،_0', 'asp': 'na', 'cas': 'u', 'gen': 'm', 'mod': 'na', 'num': 's', 'per': 'na', 'stt': 'i', 'vox': 'na', 'pos': 'noun_prop', 'prc0': '0', 'prc1': '0', 'prc2': '0', 'prc3': '0', 'enc0': '0'}, 'فاطمه': {'bw': 'fa/CONJ+>uTum/NOUN+a/CASE_DEF_ACC+hu/POSS_PRON_3MS', 'gloss': 'blockhouse;pillbox', 'diac': 'فَأُطُمَهُ', 'lemma': 'أُطُم_1', 'asp': 'na', 'cas': 'a', 'gen': 'm', 'mod': 'na', 'num': 's', 'per': 'na', 'stt': 'c', 'vox': 'na', 'pos': 'noun', 'prc0': '0', 'prc1': '0', 'prc2': 'fa_conj', 'prc3': '0', 'enc0': '3ms_poss'}, '.': {'bw': './PUNC', 'gloss': '.', 'diac': '.', 'lemma': 

In [39]:
# testanalyses[0].analysis

In [6]:
# testanalyses[0].analysis[testanalyses[0].words[2]]

In [15]:
locmap = translit_rules.load_loc_mappings()
exceptional = translit_rules.load_exceptional_spellings()
# translit_rules.translit_simple(testanalyses[0].words[0],locmap,exceptional)

In [121]:
# testsents = list(dev[dev['sentID']==27774]['ar'].values)
testsents = ['رايت كلبة']
testsents = [tokenize_skiphyph(sent) for sent in testsents]
inputconfig = generate_analyser_input(testsents)
outxml = analyse_server(inputconfig)
testanalyses = list(parse_analyser_output(outxml))



In [120]:
pd.DataFrame([{'from':x[0],'to':x[1],'freq':y} for x,y in logger.morph_rules.items()]).sort_values('freq',ascending=False)

,from,to,freq
0,case-ending,removed,3
1,index-0 capitalize,capitalized,1
2,adj/nounprop capitalized gloss,capitalized,1


In [118]:
logger.exceptional_rules

{('الله', 'اللَاه±'): 1}

In [122]:
logger = translit_rules.rules_logger
translit_rules.translit_morph(testanalyses[0],locmap,exceptional,logger=logger)

'Raʼayt kalbat'

In [56]:
[{'keys':x,'values':y} for x,y in logger.chunks.items()]

[{'keys': ('<<<<<', ''), 'values': 4},
 {'keys': ('NOMAP', "['"), 'values': 1},
 {'keys': ('ك', 'k'), 'values': 1},
 {'keys': ('ت', 't'), 'values': 3},
 {'keys': ('ا', 'ā'), 'values': 1},
 {'keys': ('ب', 'b'), 'values': 1},
 {'keys': ('>>>>>', ''), 'values': 7},
 {'keys': ('<<<<<ا', 'a'), 'values': 3},
 {'keys': ('ل', 'l'), 'values': 4},
 {'keys': ('د', 'd'), 'values': 1},
 {'keys': ('ر', 'r'), 'values': 1},
 {'keys': ('ن', 'n'), 'values': 1},
 {'keys': ('ظ', 'ẓ'), 'values': 1},
 {'keys': ('ي', 'y'), 'values': 4},
 {'keys': ('م', 'm'), 'values': 2},
 {'keys': ('ف', 'f'), 'values': 1},
 {'keys': ('س', 's'), 'values': 1},
 {'keys': ('ه', 'h'), 'values': 1},
 {'keys': ('ق', 'q'), 'values': 1},
 {'keys': ('و', 'w'), 'values': 1},
 {'keys': ('NOMAP', ":']"), 'values': 1}]

In [73]:
pd.DataFrame([{'from':x[0],'to':x[1],'freq':y} for x,y in logger.chunks.items()])

,from,to,freq
0,<<<<<,,4
1,NOMAP,[',1
2,ك,k,1
3,ت,t,3
4,ا,ā,1
5,ب,b,1
6,>>>>>,,7
7,<<<<<ا,a,3
8,ل,l,4
9,د,d,1


In [68]:
[{'f': f,'t':t} for f,t in logger.chunks.keys()]

[{'f': '<<<<<', 't': ''},
 {'f': 'NOMAP', 't': "['"},
 {'f': 'ك', 't': 'k'},
 {'f': 'ت', 't': 't'},
 {'f': 'ا', 't': 'ā'},
 {'f': 'ب', 't': 'b'},
 {'f': '>>>>>', 't': ''},
 {'f': '<<<<<ا', 't': 'a'},
 {'f': 'ل', 't': 'l'},
 {'f': 'د', 't': 'd'},
 {'f': 'ر', 't': 'r'},
 {'f': 'ن', 't': 'n'},
 {'f': 'ظ', 't': 'ẓ'},
 {'f': 'ي', 't': 'y'},
 {'f': 'م', 't': 'm'},
 {'f': 'ف', 't': 'f'},
 {'f': 'س', 't': 's'},
 {'f': 'ه', 't': 'h'},
 {'f': 'ق', 't': 'q'},
 {'f': 'و', 't': 'w'},
 {'f': 'NOMAP', 't': ":']"}]

In [14]:
testanalyses[0].words


['كتاب', 'الدر', 'النظيم', 'في', 'تسهيل', 'التقويم', ':']

In [4]:

translit_rules.translit_morph(testanalyses[0],locmap,exceptional)

'Kitāb al-durr al-Nẓym fī tashīl al-taqwīm :'

In [37]:
deveval = pd.read_csv(f'{home}/evaluation/dev/rules-morph.tsv',delimiter='\t')

# odd analysis for li prep
#  'لنباتات': {'bw': 'la/PREP+nabAt/NOUN+At/NSUFF_FEM_PL+i/CASE_DEF_GEN',


In [25]:
deveval[deveval['sentID']==8915] 

,gold_tok,source_tok,aligned_source/gold,predict_1_tok,sentID,recID,comb.tag,aligned_source/prediction,match,match_ignore_case
2359,Ibn,لأبي,False,la,8915,umich-1141,100a,False,False,False
2360,Qudāmah,محمد,False,-,8915,umich-1141,100a,False,False,False
2361,"al-Maqdisī,",عبد,False,Abī,8915,umich-1141,100a,False,False,False
2362,Muḥammad,الله,False,Muḥammad,8915,umich-1141,100a,False,True,True
2363,ibn,بن,False,ʻAbd,8915,umich-1141,100a,False,False,False
2364,"Aḥmad,",أحمد,False,Allāh,8915,umich-1141,100a,False,False,False
2365,NaN,بن,False,ibn,8915,umich-1141,100a,False,False,False
2366,NaN,قدامة,False,Aḥmad,8915,umich-1141,100a,False,False,False
2367,NaN,المقدسي.,False,ibn,8915,umich-1141,100a,False,False,False
2368,NaN,NaN,False,quddāmahu,8915,umich-1141,100a,False,False,False


In [43]:
nonaligned = deveval[deveval['aligned_source/prediction']==False]['sentID'].drop_duplicates()
deveval[deveval['sentID']==nonaligned.iloc[0]] 

,gold_tok,source_tok,aligned_source/gold,predict_rules-morph_tok,sentID,recID,comb.tag,aligned_source/prediction,match,match_ignore_case
61792,"Al-Juhani,",أل-جهن،,True,al--,237799,loc-6006,100a,False,False,False
61793,Hamdi,همد,True,"jhn,",237799,loc-6006,100a,False,False,False
61794,bin,بن,True,hamad,237799,loc-6006,100a,False,False,False
61795,Hamza.,همز.,True,ibn,237799,loc-6006,100a,False,False,False
61796,NaN,NaN,True,Hamz.,237799,loc-6006,100a,False,False,False
